In [1]:
import os
import json
import logging 

os.environ["CR_API_AGENT"] = "S2ORC metadata extraction; mailto:hcssteamukraine@gmail.com"
os.environ["CR_API_MAILTO"] = "hcssteamukraine@gmail.com"

In [2]:
import pandas as pd
import crossref_commons.retrieval

In [4]:
from helpers import read_jsonl, texts_path, meta_path

In [4]:
def create_logger(name: str):
    """Create logger with DEBUG level & stream handler."""
    logger = logging.getLogger(name)
    logger.setLevel(logging.DEBUG)
    sh = logging.StreamHandler()
    sh.setLevel(logging.DEBUG)
    formatter = logging.Formatter("%(asctime)s [%(levelname)s]: %(message)s")
    sh.setFormatter(formatter)
    logger.addHandler(sh)
    return logger


logger = create_logger("crossref")

In [5]:
relevant_papers = {paper["paper_id"] for paper in read_jsonl(texts_path)}

In [5]:
metadata = []
for paper in read_jsonl(meta_path):
    _id = paper["paper_id"]
    date_published = paper["year"]
    if _id in relevant_papers:
        metadata.append({
            "id": _id,
            "DOI": paper["doi"],
            "year": date_published
        })

NameError: name 'relevant_papers' is not defined

In [7]:
metadata[0]

{'id': '157553241', 'DOI': '10.4000/rfcb.1364', 'year': 2017}

---

In [8]:
def retrieve_crossref_data(dois):
    for doi in dois:
        try:
            data = crossref_commons.retrieval.get_publication_as_json(doi)
        except ValueError as ex:
            data = {"DOI": doi, "Error": str(ex)}
        yield {"DOI": doi, **data}

In [9]:
DOIs = {paper["DOI"] for paper in metadata if paper["DOI"] is not None}

In [14]:
counter = 0
logger.info("START")
with open("../data/processed/dois-metadata.jsonl", "w", encoding="utf-8") as out:
    for result in retrieve_crossref_data(DOIs):
        json.dump(result, out)
        out.write("\n")
        counter += 1
        if counter % 500 == 0:
            logger.info(f"Downloaded {counter} entries.")
            
logger.info("FINISH")

2021-08-18 23:34:31,002 [INFO]: START
INFO:crossref:START
2021-08-18 23:38:20,688 [INFO]: Downloaded 500 entries.
INFO:crossref:Downloaded 500 entries.
2021-08-18 23:42:09,784 [INFO]: Downloaded 1000 entries.
INFO:crossref:Downloaded 1000 entries.
2021-08-18 23:45:59,573 [INFO]: Downloaded 1500 entries.
INFO:crossref:Downloaded 1500 entries.
2021-08-18 23:49:50,133 [INFO]: Downloaded 2000 entries.
INFO:crossref:Downloaded 2000 entries.
2021-08-18 23:53:38,977 [INFO]: Downloaded 2500 entries.
INFO:crossref:Downloaded 2500 entries.
2021-08-18 23:57:30,175 [INFO]: Downloaded 3000 entries.
INFO:crossref:Downloaded 3000 entries.
2021-08-19 00:01:21,052 [INFO]: Downloaded 3500 entries.
INFO:crossref:Downloaded 3500 entries.
2021-08-19 00:05:12,918 [INFO]: Downloaded 4000 entries.
INFO:crossref:Downloaded 4000 entries.
2021-08-19 00:09:02,795 [INFO]: Downloaded 4500 entries.
INFO:crossref:Downloaded 4500 entries.
2021-08-19 00:12:51,633 [INFO]: Downloaded 5000 entries.
INFO:crossref:Downloade